In [9]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#model_base_1_3 = AutoModelForCausalLM.from_pretrained("EleutherAI/polyglot-ko-1.3b")
#odel_base_1_3.to(device)

#model_base = AutoModelForCausalLM.from_pretrained("EleutherAI/polyglot-ko-5.8b")
#model_base.to(device)

model_dapt = AutoModelForCausalLM.from_pretrained('/workspace/ElementGPT/experiments/poly5.8b-DAPT')
model_dapt.to(device)

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/polyglot-ko-5.8b")



In [6]:
import koco
test = koco.load_dataset('korean-hate-speech', mode='test')
type(test)
print(test[0])

{'comments': 'ㅋㅋㅋㅋ 그래도 조아해주는 팬들 많아서 좋겠다 ㅠㅠ 니들은 온유가 안만져줌 ㅠㅠ', 'news_title': '"샤이니 온유, 클럽 강제추행 \'무혐의\' 처분 받았다"'}


In [11]:
def calculate_perplexity(model, context, target):
    # Create the input with both context and target
    input_text = context + " " + target
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)

    # Create the labels by only encoding the target and filling rest with -100
    labels = tokenizer.encode(target, return_tensors='pt').to(device)
    labels = labels.to(device)
    padded_labels = -100 * torch.ones_like(input_ids)
    padded_labels[:, -labels.size(1):] = labels

    with torch.no_grad():
        outputs = model(input_ids, labels=padded_labels)
        loss = outputs.loss
    return torch.exp(loss)

perplexity_list = []
for data in test:
    news_title = data['news_title']
    context = f"뉴스 제목: {news_title}" + f"댓글: "
    target = data['comments']
    #perplexity = calculate_perplexity(model= model_base, context=context, target= target)
    perplexity = calculate_perplexity(model= model_dapt, context=context, target=target)
    perplexity_list.append(perplexity.item())

#print(f"Average perplexity score for original polyglot-ko-5.8b model: {sum(perplexity_list) / len(perplexity_list)}")
print(f"Average perplexity score for polyglot-ko-5.8b-w_DAPT model: {sum(perplexity_list) / len(perplexity_list)}")


Average perplexity score for polyglot-ko-5.8b-w_DAPT model: 1304.034141183634


In [8]:
perplexity_list = []
for data in test:
    news_title = data['news_title']
    context = f"뉴스제목: {news_title}" + f"댓글: "
    target = data['comments']
    perplexity = calculate_perplexity(model= model_base, context=context, target= target)
    #perplexity = calculate_perplexity(model= model_dapt, context=context, target=target)
    perplexity_list.append(perplexity.item())

print(f"Average perplexity score for original polyglot-ko-5.8b model: {sum(perplexity_list) / len(perplexity_list)}")
#print(f"Average perplexity score for polyglot-ko-5.8b-w_DAPT model: {sum(perplexity_list) / len(perplexity_list)}")

Average perplexity score for original polyglot-ko-5.8b model: 303.0976689180065


In [8]:
perplexity_list = []
for data in test:
    news_title = data['news_title']
    context = f"뉴스제목: {news_title}" + f"댓글: "
    target = data['comments']
    perplexity = calculate_perplexity(model= model_base_1_3, context=context, target= target)
    #perplexity = calculate_perplexity(model= model_dapt, context=context, target=target)
    perplexity_list.append(perplexity.item())

print(f"Average perplexity score for original polyglot-ko-1.3b model: {sum(perplexity_list) / len(perplexity_list)}")
#print(f"Average perplexity score for polyglot-ko-5.8b-w_DAPT model: {sum(perplexity_list) / len(perplexity_list)}")

Average perplexity score for original polyglot-ko-1.3b model: 635.0089889598333


In [4]:
def data_tokenize(data_list, tokenizer):
    tokenized_inputs = []
    for data in data_list:
        comments = data.get('comments', '')
        title = data.get('news_title', '')
        prompt1 = f'뉴스 제목: {title} '
        prompt2 = f'댓글: {comments}'
        input_sequence = prompt1 + prompt2
        inputs = tokenizer.encode(input_sequence, return_tensors = 'pt')
        tokenized_inputs.append(inputs)
    return tokenized_inputs

tokenized_input = data_tokenize(test, tokenizer=tokenizer)
print((tokenized_input)[0])

def calculate_log_likelihood(model, input, device):
    input = input.to(device)
    model.eval()
    with torch.no_grad():
        outputs = model(input, labels=input)
        return outputs.loss.item()



def calculate_perplexity(model, input):
    model.eval()
    with torch.no_grad():
        output = model(input)
        logits = output.logits
        print(logits)
        
        prompt2_start = input[0].tolist().index(tokenizer.encode(tokenizer.eos_token)[0]) +3

        logits_prompt2 = logits[..., prompt2_start:,:]
        print(logits_prompt2)
        p = torch.nn.functional.softmax(logits_prompt2, dim=-1)

        log_p = torch.log(p)
        print(log_p)
        nll_loss = torch.nn.functional.nll_loss(torch.log(p), logits_prompt2[:, 0, :].long())
        # nll_loss = -torch.gather(log_p,2,input[..., prompt2_start:,:]).mean()
        perplexity_prompt2 = torch.exp(nll_loss)

    return perplexity_prompt2.item()


        

tensor([[ 4883,  4526,    29,   490,  5213, 22532,  1148,   608,    15,  5240,
          4002,  7554,   514,   529, 13713,    10,  4861,   719,   622,   267,
             5,  5952,    29, 10609,   237,  9664,   237,  9664,   237,  9664,
           237,  3791,   534,   399, 29143,   272,  2335,   355,   750,  2341,
           909, 16471, 10609,   258,  9664,   258,  3211,   355,   296,  1148,
          8949,   596,   380,   970,  6973, 10609,   258,  9664,   258]])


In [15]:
sample = data_tokenize(test, tokenizer= tokenizer)[3]
calculate_log_likelihood(model= model_base, input = sample, device=device)

calculate_perplexity(model=model_base, input=sample.to(device))

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.